In [1]:
import requests
import json
import pandas as pd
import time
#import convex as cx
import tmqa34 as tmqa

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [2]:
#f_test = open("/data/users/romain.claret/tm/wikidata-simplequestions/annotated_wd_data_test.txt",'r')
#out_test = f_test.readlines()

In [3]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [4]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [5]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [6]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
#ask_convex("100% senorita is a television show in what language?")

In [7]:
def ask_tmqa(question, verbose=False, timer=False, show_graph=False, cores=tmqa.mp.cpu_count(), banning_str=False):
    tmqa_result = tmqa.answer_question(question, verbose=verbose, timer=timer, show_graph=show_graph, cores=cores, banning_str=banning_str)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    if tmqa_result == (False,False):
        return False
    #if tmqa_result[]
    #if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
    #    return False
    return tmqa_result[0][0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#answer = ask_tmqa("Who is the wife of Barack Obama")
#print(answer)

In [8]:
# Evaluate qanswer
#arguments[0] Subject
#arguments[1] Predicate
#arguments[2] Object
#arguments[3] question

#HEADERS = ['question', 'source', 'qanswer', 'platypus', 'convex', 'tmqa']
#df = pd.DataFrame(columns=HEADERS)

#start_time = time.time()
#out_test_len = len(out_test)
##print(out_test_len)
#for i, line in enumerate(out_test):
#    arguments = line.split("\t")
#    print("\r\t>>> Processing {}/{}: {}".format(i,out_test_len,arguments[3][:-1]), end='                                  ')
#    if i < len(df):
#        continue
#    result_qanswer = ask_qanswer(arguments[3][:-1])
#    result_platypus = ask_platypus(arguments[3][:-1])
#    result_convex = ask_convex(arguments[3][:-1])
#    #result_tmqa = ask_tmqa(arguments[3][:-1])[1]
#    print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == arguments[2] else False
#    print("\n",i, "-> Platypus", result_platypus) if result_platypus == arguments[2] else False
#    print("\n",i, "-> Convex", result_convex) if result_convex == arguments[2] else False
#    #print("\n",i, "-> TMqa", result_tmqa) if result_tmqa == arguments[2] else False
#    df = df.append({"question":arguments[3][:-1], 'source':arguments[2], 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex, 'tmqa1':result_tmqa}, ignore_index=True)
#
#end_time = time.time()
#print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
#print(df)

In [9]:
#len(df)

In [10]:
#df.tail(5)

In [11]:
#SAVING
#pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(len(df_loaded)))

In [12]:
#LOADING
#df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-9961.pickle.bz2")

#/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-5.pickle.bz2


In [13]:
#df = df_loaded

In [14]:
#df_loaded

In [15]:
#df_loaded = df_loaded.replace("", False)

In [16]:
#df_loaded['qanswer'][34] = False

In [17]:
#del df_loaded['tm2']

In [18]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [19]:
#df_loaded['tm1_top4'] = "False"

In [20]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [21]:
#df_loaded

In [22]:
#LOADING QAGraph

df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-1875.pickle.bz2")



In [23]:
print(df_loaded.dtypes)

question      object
source        object
qanswer       object
platypus      object
convex        object
tm1           object
tm1_time      object
tm1_top2      object
tm1_top3      object
tm1_top4      object
tm1_top5      object
tm1_topall    object
dtype: object


In [ ]:
banning_str = False#[["ř","r"],["ø","o"]]

df_loaded_len = len(df_loaded)
for i, question in enumerate(df_loaded['question']):
    if i >= 1875:
        source = str(df_loaded['source'][i])
        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
        
        start_time = time.time()
        result_tmqa = ask_tmqa(question, verbose=True, timer=True, banning_str=banning_str)
        
        if result_tmqa:
            df_loaded['tm1'][i] = result_tmqa[0]
            if source in result_tmqa[:2]:
                df_loaded['tm1_top2'][i] = True
            if source in result_tmqa[:3]:
                df_loaded['tm1_top3'][i] = True
            if source in result_tmqa[:4]:
                df_loaded['tm1_top4'][i] = True
            if source in result_tmqa[:5]:
                df_loaded['tm1_top5'][i] = True
            if source in result_tmqa:
                df_loaded['tm1_topall'][i] = True
        else:
            df_loaded['tm1'][i] = False
        end_time = time.time()
        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa:",str(source))
        
        pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(i))
        
        print(df_loaded.loc[i,:])
        print("\n")

1490/9961 Which war did William J. Obrien fight in? -> source: Q362
User input: Which war did William J. Obrien fight in?
--> Auto correcting question in progress...

-> Auto corrected q_nlp: Which war did William J. Obrien fight in 
-> q_themes: ([(War, ['Q198', 'Q331811']), (war, ['Q26883397'])], [Which war, William J. Obrien, Which war did William, J. Obrien, Did William J. Obrien Fight, william j. obrien, which War])
-> q_themes_enhanced: [('William', ['Q129987']), ('J.', ['Q19803507']), ('Obrien', ['Q37124748']), ('Fight', ['Q11331861']), ('fight', ['Q25965834'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: fight
-> q_predicates: [(did, ['P248']), (fight, [])]
-> q_predicates 	Running time is 17.55s
--> Potential meaningful keywords for the sentence: ['War', 'war', 'William', 'J.', 'Obrien', 'Fight', 'fight']
q_focused_parts: [(war, ['Q26883397', 'Q198', 'P607'])]
-> Building the graph with k_deep 3 .

-> q_themes_enhanced: [('produce', ['Q11336088']), ('Produce', ['Q11336090'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
-> q_predicates: [(produced, ['P162', 'P2849'])]
-> q_predicates 	Running time is 5.44s
--> Potential meaningful keywords for the sentence: ['hands', 'Hands', 'produce', 'Produce']
q_focused_parts: [(hands, ['Q25505093', 'Q773489'])]
-> Building the graph with k_deep 3 ... (could be long)
->New graph 	Running time is 20.81s
-->  34 nodes and 32 edges
--> Removing meaningless subgraphs
--> New graph of: 34 nodes and 32 edges
---> Rebuilding the graph with k_deep 6 ... Previously: 34 nodes or 32 edges was below the limit of 100
->New graph 	Running time is 20.0s
-->  46 nodes and 44 edges
--> Removing meaningless subgraphs
--> New graph of: 46 nodes and 44 edges
---> Rebuilding the graph with k_deep 8 ... Previously: 46 nodes or 44 edges was below the limit of 100
->New graph 	Running time is 19.49s
-->  51 nodes and 50 edges

--> hypothesises: [['Q27050436', 0.27278627748966766], ['Q11424', 0.23862869552695506]]
->Computing hypothesises 	Running time is 1.45s
-> Computing golden paths...
--> len(golden_paths): 1
->	Running time is 4.36s
--> len(cleared_golden_paths): 1
---> First path: ['Q27050436', 'P31', 'Q11424', 'P31', 'Q14947645']
->	Total Running time is 481.36s

->	Running time is 481.64s
False ---> result_tmqa: Q2352011
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-1493.pickle.bz2
question      Name an acoustic music artist.
source                              Q2352011
qanswer                                Q1203
platypus                               False
convex                               Q829727
tm1                                Q27050436
tm1_time                              481.64
tm1_top2                               False
tm1_top3                               False
tm1_top4                           

->New graph 	Running time is 21.34s
-->  96 nodes and 96 edges
--> Removing meaningless subgraphs
--> New graph of: 96 nodes and 96 edges
---> Loop detected, returning the graph in the current state
-> predicates_dict: {'P373': 1, 'P1269': 3, 'P1479': 1, 'P1552': 1, 'P21': 1, 'P3744': 1, 'P407': 1, 'P3984': 1, 'P1478': 1, 'P805': 2, 'P1343': 2, 'P518': 1, 'P571': 1, 'P131': 1, 'P735': 1, 'P580': 1, 'P582': 1, 'P166': 1, 'P31': 10, 'P1412': 1, 'P937': 1, 'P569': 1, 'P495': 1, 'P642': 1, 'P279': 1, 'P576': 1, 'P17': 1, 'P5008': 1, 'P361': 1, 'P625': 1, 'P910': 1, 'P50': 2, 'P127': 1, 'P276': 1}
-> paths_keywords: (['sex', 'paul ekins'], {'instance of': [instance of, ['P31']], 'sex or gender': [sex or gender, ['P21']], 'Sex': [sex or gender, ['P21']]}, [What])
-> Computing possible paths... (could be long)
--> len(path_nodes): 282
->Computing possible paths 	Running time is 12.71s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 278
->	Running time is 4.18s
-> Computin

-> paths_keywords: (['star', 'hohmann', 'leading actor'], {}, [What])
-> Computing possible paths... (could be long)
--> len(path_nodes): 4
->Computing possible paths 	Running time is 18.72s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 4
->	Running time is 3.88s
-> Computing hypothesises...
--> hypothesises: []
->Computing hypothesises 	Running time is 0.15s
-> Looping on aggressive mode...

Looped in aggressive mode with: what star does 9661 hohmann orbit
--> Auto correcting question in progress...
-> Auto corrected q_nlp: What star does 9661 Hohmann orbit 
-> q_themes: ([(star, ['Q523', 'Q12068953']), (Hohmann, ['Q21508278', 'Q5877660']), (Star, ['Q10852461', 'Q1082758'])], [star does Hohmann])
-> q_themes_enhanced: []
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: star
behold: get_most_similar started with: do
-> q_predicates: [(does, []), (star, []), (orbit, ['P522', 'P2045'])]
-> 

...

--> len(golden_paths): 3
->	Running time is 75.29s
--> len(cleared_golden_paths): 1
---> First path: ['Q44823', 'P131', 'Q1190728', 'P585', '2010-01-01T00:00:00Z']
->	Total Running time is 185.53s

->	Running time is 185.83s
True ---> result_tmqa: Q44823
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-1694.pickle.bz2
question      where is the las colonias department in 
source                                          Q44823
qanswer                                          False
platypus                                         False
convex                                           False
tm1                                             Q44823
tm1_time                                        185.83
tm1_top2                                          True
tm1_top3                                          True
tm1_top4                                          True
tm1_top5                                     

--> len(golden_paths): 77
->	Running time is 79.48s
--> len(cleared_golden_paths): 38
---> First path: ['Q4104520', 'P136', 'Q483394', 'P31', 'Q7643432']
->	Total Running time is 233.58s

->	Running time is 233.9s
False ---> result_tmqa: Q343568
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-1696.pickle.bz2
question      Which genre is disneys villains revenge videogame
source                                                  Q343568
qanswer                                                   False
platypus                                                  False
convex                                                Q21824803
tm1                                                    Q4104520
tm1_time                                                  233.9
tm1_top2                                                  False
tm1_top3                                                  False
tm1_top4                    

--> len(golden_paths): 509
->	Running time is 676.87s
--> len(cleared_golden_paths): 417
---> First path: ['Q258', 'P17', 'Q6256', 'P31', 'Q159', 'P27', 'Q400825', 'P585', '1945-01-01T00:00:00Z']
->	Total Running time is 1081.89s

->	Running time is 1082.22s
False ---> result_tmqa: Q159
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-1698.pickle.bz2
question      what country is georgi yumatov a citizen of
source                                               Q159
qanswer                                            Q15180
platypus                                            False
convex                                               Q159
tm1                                                  Q258
tm1_time                                          1082.22
tm1_top2                                            False
tm1_top3                                             True
tm1_top4                                

-> q_predicates: [(be, ['P31']), (Asian, ['P1386']), (language, ['P1412', 'P364']), (TV, ['P449']), (series, ['P179', 'P1545']), (beloved, []), (spoken, ['P37', 'P989'])]
-> q_predicates 	Running time is 69.27s
--> Potential meaningful keywords for the sentence: ['Asian', 'TV series', 'language', 'Language', 'spoken', 'Beloved', 'Spoken', 'My Beloved']
q_focused_parts: [(language, ['P1412', 'Q315', 'Q34770']), (Asian, ['Q4806537', 'Q7974'])]
-> Building the graph with k_deep 3 ... (could be long)
->New graph 	Running time is 22.84s
-->  63 nodes and 60 edges
--> Removing meaningless subgraphs
--> New graph of: 53 nodes and 52 edges
---> Rebuilding the graph with k_deep 5 ... Previously: 53 nodes or 52 edges was below the limit of 100
->New graph 	Running time is 23.13s
-->  75 nodes and 72 edges
--> Removing meaningless subgraphs
--> New graph of: 63 nodes and 62 edges
---> Rebuilding the graph with k_deep 7 ... Previously: 63 nodes or 62 edges was below the limit of 100
->New graph 	R

->	Running time is 196.12s
False ---> result_tmqa: Q1297
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-1701.pickle.bz2
question      what city did john e. smith die
source                                  Q1297
qanswer                                 False
platypus                                False
convex                                  Q1297
tm1                                  Q1006668
tm1_time                               196.12
tm1_top2                                False
tm1_top3                                False
tm1_top4                                False
tm1_top5                                False
tm1_topall                              False
Name: 1701, dtype: object


1702/9961 what is buddy jackson's position? -> source: Q674953
User input: what is buddy jackson's position?
--> Auto correcting question in progress...
-> Auto corrected q_nlp: What is Buddy Jackson position 
-> 

In [25]:
df_loaded.head(1875)

,question,source,qanswer,platypus,convex,tm1,tm1_time,tm1_top2,tm1_top3,tm1_top4,tm1_top5,tm1_topall
0,Which genre of album is harder.....faster?,Q1096120,False,Q5487302,False,Q4104520,90.13,False,False,False,False,False
1,what city was alex golfis born in,Q1524,Q12751,False,Q83065,Q445559,69.08,False,False,False,False,False
2,what film is by the writer phil hay?,Q192979,False,False,Q1337306,Q226730,64.27,False,False,False,False,False
3,Where did roger marquis die,Q1637790,False,False,Q1637790,Q1637790,141.59,True,True,True,True,True
4,what was the cause of death of yves klein,Q12152,Q12152,False,Q12152,Q90,223.62,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1870,audrey williams performs which genre of music?,Q83440,Q83440,False,False,Q4413495,129.76,False,False,False,False,False
1871,What is the name of an album that is apart of ...,Q1341076,False,False,False,False,127.04,False,False,False,False,False
1872,which language is spoken in temmink: the ultim...,Q7411,Q7411,False,Q55,Q7850,186.92,False,False,False,False,False
1873,what kind of film genre is the movie der 20. ...,Q130232,False,False,False,False,121.91,False,False,False,False,False


In [ ]:
df_copy = df_loaded.copy()
#df = df.replace("", False)

In [ ]:
max_row = 1164
df_len = len(df_copy)
df_qanswer_max = df_copy[(df_copy.index<=max_row) & (df_copy.qanswer == df_copy.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df_copy[(df_copy.index<=max_row) & (df_copy.platypus == df_copy.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df_copy[(df_copy.index<=max_row) & (df_copy.convex == df_copy.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df_copy[(df_copy.index<=max_row) & (df_copy.tm1 == df_copy.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df_copy[(df_copy.index<=max_row) & (df_copy.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
print("convex:", df_convex_max_len, df_convex_max_len/max_row)
print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

In [ ]:
print("qanswer:",len(df_copy[df_copy.qanswer == df_copy.source]),len(df_copy[df_copy.qanswer == df_copy.source])/len(df_copy))
print("platypus:",len(df_copy[df_copy.platypus == df_copy.source]),len(df_copy[df_copy.platypus == df_copy.source])/len(df_copy))
print("convex:",len(df_copy[df_copy.convex == df_copy.source]),len(df_copy[df_copy.convex == df_copy.source])/len(df_copy))
print("tm1:",len(df_copy[df_copy.tm1 == df_copy.source]),len(df_copy[df_copy.tm1 == df_copy.source])/len(df_copy))
print("tm1_top2:",len(df_copy[df_copy.tm1_top2 == True]),len(df_copy[df_copy.tm1_top2 == True])/len(df_copy))
print("tm1_top3:",len(df_copy[df_copy.tm1_top3 == True]),len(df_copy[df_copy.tm1_top3 == True])/len(df_copy))
print("tm1_top4:",len(df_copy[df_copy.tm1_top4 == True]),len(df_copy[df_copy.tm1_top4 == True])/len(df_copy))
print("tm1_top5:",len(df_copy[df_copy.tm1_top5 == True]),len(df_copy[df_copy.tm1_top5 == True])/len(df_copy))
print("tm1_topall:",len(df_copy[df_copy.tm1_topall == True]),len(df_copy[df_copy.tm1_topall == True])/len(df_copy))



In [ ]:
#df.tm1_top2 = df.tm1_top3